In [88]:
import pandas as pd
from pathlib import Path

INVOICE_DIR = '/work/data/OpenIRIS/BIU-20200916'
INVOICE_NAME = 'Invoice18_fixed.xlsx'

INVOICE_DIR = Path(INVOICE_DIR)
if not INVOICE_DIR.is_dir():
    raise ValueError('Please check INVOICE_DIR.')
INVOICE_FILE = INVOICE_DIR / INVOICE_NAME
if not INVOICE_FILE.exists():
    raise ValueError('Please check INVOICE_FILE.')

ERITTELYT = INVOICE_DIR / 'erittelyt'
ERITTELYT.mkdir(exist_ok=True)

df = pd.read_excel(INVOICE_FILE)

In [89]:
WBS = 'Cost center code'
WBSs = df[WBS].unique()

In [90]:
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font

wb = Workbook()

df.sort_values(by=['Group', WBS], inplace=True)

begin = df['Booking start'].min().date()
end = df['Booking end'].max().date()

cols = ['Booking start', 'Booking end', 'Resource', 'User name', 'Charge', \
        'Qty', 'Price (detailed)', 'Request Title', 'Discount', 'Product comments', \
        'Comments (charge)',]

def prepare_sheet(ws, tmp):
    PI = tmp['Group'][0]
    email = tmp['Group head(s) text'][0]
    wbs = tmp['Cost center code'][0]
    profit_center = tmp['Remit code'][0]
    organization = tmp['Organization'][0]
    total_sum = tmp['Charge'].sum()
    
    ws.title = PI + ' ' + str(wbs)
    
    ws['C1'] = 'Light Microscopy Unit'
    ws['C3'] = 'Billing cycle:'
    ws['D3'] = str(begin) + ' - ' + str(end)
    ws['C4'] = 'Group/Payer:'
    ws['D4'] = PI
    ws['C5'] = 'Email address:'
    ws['D5'] = email
    ws['C6'] = 'Billing address:'
    ws['D6'] = ''
    ws['C7'] = 'Organization:'
    ws['D7'] = organization
    ws['C8'] = 'Cost center:'
    ws['D8'] = wbs
    ws['C9'] = 'Profit center:'
    ws['D9'] = profit_center
    ws['C10'] = 'Total sum:'
    ws['D10'] = total_sum
    ws['D10'].number_format = '#,##0.00€'
    ws['D10'].font = Font(bold=True)
    ws['D11'] = ''
    ws['D12'] = ''
    
    tmp = tmp[cols]
    for r in dataframe_to_rows(tmp, index=False, header= True):
        ws.append(r)
        
    ws.column_dimensions['A'].width = 25
    ws.column_dimensions['B'].width = 25
    ws.column_dimensions['C'].width = 25
    ws.column_dimensions['D'].width = 25
    ws.column_dimensions['E'].width = 10
    ws.column_dimensions['F'].width = 10
    ws.column_dimensions['F'].width = 10
    ws.column_dimensions['G'].width = 40
    ws.column_dimensions['H'].width = 40
    ws.column_dimensions['I'].width = 10
    ws.column_dimensions['J'].width = 30
    ws.column_dimensions['K'].width = 30
    
    ws['C1'].font = Font(bold=True)
    for cell in ws['13']:
        cell.font = Font(bold=True)
    
    for cell in ws['E']:
        cell.number_format = '###0.00'
   
for wbs in WBSs:
    tmp = df[df[WBS] == wbs]
    tmp = tmp.reset_index()
    
    #print(wbs)
    if tmp['Group'].size == 0:
        print('no group found')
        continue
    
    # add new sheet to main workbook
    ws = wb.create_sheet(PI + ' ' + str(wbs))
    prepare_sheet(ws, tmp)

    # create separate workbook per wbs
    wb2 = Workbook()
    ws2 = wb2.active
    prepare_sheet(ws2, tmp)
    wb2.save(ERITTELYT / (ws2.title + '.xlsx'))
    
wb.save(INVOICE_DIR / (INVOICE_FILE.stem + '_split.xlsx'))
    
    

no group found
